In [1]:
import os
# from imutils import paths
import random
from shutil import copyfile
# import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
datapath = r"F:\Deep Learning\Facial Recognision\Facial Recognesion"
train_datapath = r"F:\Deep Learning\Facial Recognision\Facial Recognesion\Training"
validation_datapath = r"F:\Deep Learning\Facial Recognision\Facial Recognesion\Testing"

In [3]:
train_datagen = ImageDataGenerator(rescale = 1/255)
validation_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    train_datapath,
    color_mode='grayscale',
    class_mode = 'categorical',
    target_size = (48 , 48),
    batch_size = 128,

)

validation_generator = validation_datagen.flow_from_directory(
    validation_datapath , 
    batch_size = 32,
    target_size = (48 , 48),
    color_mode='grayscale',
    class_mode = 'categorical'
)


Found 28273 images belonging to 6 classes.
Found 7067 images belonging to 6 classes.


In [4]:
x , y = train_generator.next()
print((x).shape)
x , y = validation_generator.next()
print((x).shape)
# plt.imshow(x)

(128, 48, 48, 1)
(32, 48, 48, 1)


In [5]:
model = tf.keras.models.Sequential(
        [tf.keras.layers.Conv2D(64 ,(3,3), activation = 'relu' , input_shape = (48 , 48 , 1)),
        tf.keras.layers.Conv2D(64 ,(3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(.25),
    
        tf.keras.layers.Conv2D(128 ,(3,3), activation = 'relu'),
        tf.keras.layers.Conv2D(128 ,(3,3), activation = 'relu'),
        tf.keras.layers.Conv2D(256 ,(3,3), activation = 'relu'),
        tf.keras.layers.Conv2D(256 ,(3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(.25),
    
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256 ,   activation = 'relu'),
        tf.keras.layers.Dropout(.5),
        tf.keras.layers.Dense(6 , activation = 'softmax')]
)


In [6]:
from tensorflow.keras.optimizers import Adam , SGD

In [7]:
opt = SGD(lr = 0.0001 , momentum=0.9 , decay = 1e-4/25)
model.compile(loss = 'categorical_crossentropy' , optimizer = opt ,  
             metrics = ['accuracy'])

In [8]:
checkpoint_path = "training_gray/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=10)

In [9]:
model.load_weights('fer1.h5')

In [10]:
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

  ...
    to  
  ['...']
221/221 - 17s - loss: 2.5432 - accuracy: 0.5749
Restored model, accuracy: 57.49%


In [11]:
history = model.fit_generator(
        train_generator,
#         batch_size=64,
        epochs=20,
        verbose=1,
        validation_data=validation_generator,
        callbacks=[cp_callback]
           )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 221 steps, validate for 221 steps
Epoch 1/20
 22/221 [=>............................] - ETA: 11:06 - loss: 0.0778 - accuracy: 0.9725

KeyboardInterrupt: 

In [ ]:
model.load_weights('fer.h5')
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
 model.save('my_model.h5') 